In [1]:
# Load data

import pandas as pd

data = pd.read_csv("../data/Boiler_emulator_dataset_new.csv")
data

,Fuel_Mdot,Tair,Treturn,Tsupply,Water_Mdot,Condition,Class
0,1,283,333.0,363.574744,3.0,%=0.05,Lean
1,1,283,333.0,362.349517,3.0,%=0.1,Nominal
2,1,283,333.0,361.216941,3.0,%=0.15,ExcessAir
3,1,283,333.0,360.166890,3.0,%=0.20,ExcessAir
4,1,283,333.0,359.190662,3.0,%=0.25,ExcessAir
...,...,...,...,...,...,...,...
27275,3,303,333.0,341.109962,12.0,Nominal,Nominal
27276,4,303,333.0,341.112750,12.0,Nominal,Nominal
27277,1,303,333.0,340.764696,12.5,Nominal,Nominal
27278,3,303,333.0,340.786085,12.5,Nominal,Nominal


In [2]:
# Extract classes
classes = data["Class"].unique()
classes

array(['Lean', 'Nominal', 'ExcessAir', 'Fouling', 'Scaling'], dtype=object)

In [3]:
# Explore condition (Not part of input features)

conditions = {}
for c in classes:
    conditions[c] = data[data["Class"] == c]["Condition"].unique()

conditions

{'Lean': array(['%=0.05'], dtype=object),
 'Nominal': array(['%=0.1', 'Nominal'], dtype=object),
 'ExcessAir': array(['%=0.15', '%=0.20', '%=0.25', '%=0.3', '%=0.35', '%=0.40',
        '%=0.45', '%=0.50'], dtype=object),
 'Fouling': array(['F = 0.01', 'F = 0.06', 'F = 0.11', 'F = 0.16', 'F = 0.21',
        'F = 0.26', 'F = 0.31', 'F = 0.36', 'F = 0.41', 'F = 0.46'],
       dtype=object),
 'Scaling': array(['S = 0.01', 'S = 0.06', 'S = 0.11', 'S = 0.16', 'S = 0.21',
        'S = 0.26', 'S = 0.31', 'S = 0.36', 'S = 0.41', 'S = 0.46'],
       dtype=object)}